## Anchor explanations on Adult dataset

In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the train and test set

In [3]:
nome = 'adult'
title = "../datasets/train_set_"+nome+"_strat.p"
train = open(title,"rb")
train_set = pickle.load(train)
title = "../datasets/train_label_"+nome+"_strat.p"
train_l = open(title,"rb")
train_label = pickle.load(train_l)
title = "../datasets/test_set_" + nome + "_strat.p"
test = open(title, "rb")
test_set = pickle.load(test)
title = "../datasets/test_label_" + nome + "_strat.p"
test_l = open(title, "rb")
test_label = pickle.load(test_l)

### Define and fit the model 

In [4]:
from xgboost import XGBClassifier
bb = XGBClassifier(C= 1, penalty='l2', objective='binary:logistic', seed = 42, bootstrap=True, max_depth=90,learning_rate=0.1, n_estimators=500, tree_method='auto')
bb.fit(train_set.values, train_label.values)

/Users/narets/opt/anaconda3/envs/generic_latest/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:14:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { C, bootstrap, penalty } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:14:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(C=1, base_score=0.5, booster='gbtree', bootstrap=True,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=90, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=12,
              num_parallel_tree=1, penalty='l2', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='auto', validate_parameters=1, verbosity=None)

### Define the Anchor tabular explainer
#### We pass the train set, the columns and the unique values of the labels 

In [5]:
explainer = anchor_tabular.AnchorTabularExplainer(
    train_label.unique,
    train_set.columns,
    train_set.values)

### Select one record from the test set
#### We query the model to get the prediction: in thi case it is part of class =< 50k

In [17]:
idx = 0
print(train_set.iloc[idx].values)
bb.predict(train_set.iloc[idx].values.reshape(1, -1))

[39 2 10 7 8 3 1 0 0 0 40 41]


array([0])

### We explain the record selected
#### We set the parameter threshold 

In [31]:
exp = explainer.explain_instance(train_set.iloc[idx].values.reshape(1, -1), bb.predict, threshold=0.95)

In [33]:
print('Anchor explanation: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())


Anchor explanation: education-num > 12.00 AND relationship <= 1.00 AND occupation <= 3.00 AND age > 37.00 AND marital-status <= 1.00 AND race <= 1.00 AND sex <= 0.00
Precision: 0.77
Coverage: 0.06


### We select another record, with a different prediction

In [34]:
idx = 23
print(train_set.iloc[idx].values)
bb.predict(train_set.iloc[idx].values.reshape(1, -1))

[51 3 13 1 3 1 1 0 0 0 40 41]


array([1])

In [35]:
exp = explainer.explain_instance(test_set.iloc[idx].values.reshape(1, -1), bb.predict, threshold=0.95)

In [36]:
print('Anchor explanation: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor explanation: education-num <= 9.00 AND hours-per-week <= 40.00 AND capital-gain <= 0.00 AND occupation > 3.00 AND race > 1.00
Precision: 0.99
Coverage: 0.05
